### Tabelas de partos para dashboard

#### Libs e importando os dados

In [12]:
GRUPO_PROCED = {
    'Normal/Cesário': ['0310010039', '0310010055', '0411010034', '0411010042'],
    'De Risco': ['0310010047', '0411010026']
}

In [13]:
import pandas as pd

df = pd.read_parquet('../app/datasets/outputs/partos_pe.parquet.gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83797 entries, 2622102264276 to 2622109033346
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   GESTAO     83797 non-null  int32 
 1   MUNIC_RES  83797 non-null  int32 
 2   MUNIC_MOV  83797 non-null  int32 
 3   PROC_REA   83797 non-null  object
 4   ANO_CMPT   83797 non-null  int32 
 5   MES_CMPT   83797 non-null  int32 
 6   NASC       83797 non-null  object
 7   SEXO       83797 non-null  int32 
 8   DT_INTER   83797 non-null  object
 9   DT_SAIDA   83797 non-null  object
 10  IDADE      83797 non-null  int32 
 11  MORTE      83797 non-null  object
 12  CNES       83797 non-null  object
 13  RACA_COR   83797 non-null  int32 
 14  MARCA_UTI  83797 non-null  object
 15  MARCA_UCI  83797 non-null  object
dtypes: int32(8), object(8)
memory usage: 8.3+ MB


In [14]:
df_geres = pd.read_parquet('../app/datasets/outputs/localidade_pe.parquet.gzip')
df_geres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   codmunres  185 non-null    int32 
 1   municipio  185 non-null    object
 2   cod_geres  185 non-null    int32 
 3   geres      185 non-null    object
 4   muncoddv   185 non-null    int32 
 5   geresnome  185 non-null    object
 6   microreg   85 non-null     object
dtypes: int32(3), object(4)
memory usage: 8.1+ KB


In [15]:
df_geres.head()

,codmunres,municipio,cod_geres,geres,muncoddv,geresnome,microreg
0,260005,Abreu e Lima,2601,I GERES,2600054,I GERES - Recife,MICRO I
1,260105,Araçoiaba,2601,I GERES,2601052,I GERES - Recife,MICRO I
2,260290,Cabo de Santo Agostinho,2601,I GERES,2602902,I GERES - Recife,MICRO III
3,260345,Camaragibe,2601,I GERES,2603454,I GERES - Recife,MICRO II
4,260440,Chã de Alegria,2601,I GERES,2604403,I GERES - Recife,MICRO II


In [16]:
df_tipo_parto = pd.DataFrame.from_dict(GRUPO_PROCED, orient='index').\
    T.\
    stack().\
    reset_index().\
    rename(columns={'level_1': 'TIPO_PARTO', 0: 'PROC_REA'}).\
    drop('level_0', axis=1)

df_tipo_parto

,TIPO_PARTO,PROC_REA
0,Normal/Cesário,0310010039
1,De Risco,0310010047
2,Normal/Cesário,0310010055
3,De Risco,0411010026
4,Normal/Cesário,0411010034
5,Normal/Cesário,0411010042


In [17]:
df_partos = df.reset_index().\
    merge(df_geres[['codmunres', 'municipio', 'geresnome']], how='left', left_on='MUNIC_RES', right_on='codmunres').\
    drop('codmunres', axis=1).\
    rename(columns={'geresnome': 'GERES_RES', 'municipio': 'NM_MUNIC_RES'}).\
    merge(df_geres[['codmunres', 'geresnome']], how='left', left_on='MUNIC_MOV', right_on='codmunres').\
    drop('codmunres', axis=1).\
    rename(columns={'geresnome': 'GERES_MOV', 'municipio': 'NM_MUNIC_MOV'}).\
    merge(df_tipo_parto, how='left', on='PROC_REA')

#### Export

In [18]:
df_partos[
        [
            'N_AIH', 
            'GESTAO', 
            'MUNIC_RES', 
            'MUNIC_MOV', 
            'PROC_REA', 
            'NM_MUNIC_RES', 
            'GERES_RES', 
            'GERES_MOV', 
            'TIPO_PARTO'
            ]
        ].to_parquet(
            '../app/datasets/outputs/partos_pe_reduz.parquet.gzip',
            index=False
            )

#### Tabela Geres internação para Geres residência

In [19]:
df_partos.\
    query('GESTAO == 1 and PROC_REA == ["0310010047", "0411010026"]').\
    groupby(['GERES_MOV', 'GERES_RES'], as_index=False)['N_AIH'].\
    count().\
    pivot_table(index='GERES_MOV', columns='GERES_RES', values='N_AIH', aggfunc='sum', fill_value=0, margins=True,margins_name='Total').reset_index()

GERES_RES,GERES_MOV,I GERES - Recife,III GERES - Palmares,XII GERES - Goiana,Total
0,I GERES - Recife,56,10,1,67
1,Total,56,10,1,67


#### Tabela para Mapa

In [20]:
geres_selecionada = ['I GERES - Recife', 'IV GERES - Caruaru']

In [21]:
df_partos.\
    query("GERES_MOV == @geres_selecionada and MUNIC_RES == 261160").\
    groupby(['GERES_RES', 'MUNIC_RES', 'NM_MUNIC_RES'], as_index=False)['N_AIH'].\
    count()

,GERES_RES,MUNIC_RES,NM_MUNIC_RES,N_AIH
0,I GERES - Recife,261160,Recife,10068
